In [17]:
import os
import sys
import pandas as pd
import zipfile 
import datetime
import numpy as np
from pathlib import Path
# from census import Census
# from us import states
# import zipfile 
# from configy import api_key, g_key
# from uszipcode import SearchEngine
# from geopy.geocoders import Nominatim

from geopy.geocoders import GoogleV3
from prefect.blocks.system import Secret

In [21]:
@flow()
# def api_demo_to_gcs(year: int, state: str, api_key: str) -> None:
#     dataset_state_file = f'state/{year}_states_demographic_data'
#     dataset_city_file = f'city/{year}_{state}_city_demographic_data'
#     dataset_zip_file = f'zip_code/{year}_zip_demographic_data'

#     state_df, city_df, zip_code_df = extract_demographic_data(year, state, api_key)
#     state_df, city_df, zip_code_df = transform_demo_data(state_df, city_df, zip_code_df)
#     write_demo_to_gcs(state_df, city_df, zip_code_df, dataset_state_file, dataset_city_file, dataset_zip_file)

# @flow
# def etl_demo_parent_flow() -> None:
#     years = list(range(2021, 2015, -1))
#     states_list = ['California','Florida','New York','Texas','Pennsylvania']

#     [api_demo_to_gcs(year, state, api_key) for state in states_list for year in years]

# if __name__ == '__main__':
#     etl_demo_parent_flow()

/var/folders/06/dlzbbxfn6tgfd6s50hkzfk3r0000gn/T/ipykernel_96184/2899126712.py:1: RuntimeWarning: coroutine 'Block.load' was never awaited
  api = Secret.load("api-key").get()


AttributeError: 'coroutine' object has no attribute 'get'

In [ ]:
@task(log_prints=True, tags=['extracting_RE_data'])
def extract_realestate_data() -> str:

    !kaggle datasets download -d ahmedshahriarsakib/usa-real-estate-dataset
    !unzip usa-real-estate-dataset.zip -d /opt/prefect/flows/data/real_estate/

    csv_path = os.path('/data/real_estate', 'realtor-data.csv')

    return csv_path


In [4]:
! kaggle datasets download -d ahmedshahriarsakib/usa-real-estate-dataset

 74%|████████████████████████████          | 11.0M/14.9M [00:00<00:00, 32.9MB/s]
100%|██████████████████████████████████████| 14.9M/14.9M [00:00<00:00, 33.0MB/s]


In [5]:

zip_path = './usa-real-estate-dataset.zip'
csv_folder = '/opt/prefect/flows/data/real_estate'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(csv_folder)

os.remove(zip_path)
csv_path = os.path.join(csv_folder, 'realtor-data.csv')
print(csv_path)

data/real_estate/realtor-data.csv


In [ ]:

@task(log_prints=True, tags=['Transformation'])
def transform_realestate_data(csv_path: Path, years: list, states: list) -> pd.DataFrame:

    df = pd.read_csv(csv_path)
    print(df.shape)

    df['sold_date'] = pd.to_datetime(df['sold_date'], format='%Y-%m-%d')
    df = df[df['state'].isin(states)]

    years_as_dates = [datetime.datetime(year, 1, 1) for year in years]

    df = df[df['sold_date'].between(min(years_as_dates), max(years_as_dates), inclusive=True)]

    print(df.shape)
    print(df.dtypes)

    df[['house_size','bed','bath','zip_code']] = df[['house_size','bed','bath','zip_code']].fillna(0).astype(int)

    print(df.dtypes)
    print(df.isnull().sum())
    os.remove(csv_path)
    return df 

In [ ]:
# pip install geopy

In [ ]:
# pip install uszipcode

In [ ]:
# pip install python-Levenshtein

# Extraction

In [ ]:
def extract_coordinates(year: int, state: str, api_key: str) -> pd.DataFrame:
    c = Census(api_key, year=year)
    variables = [
        'NAME',
    ]
    state_code = states.lookup(state).fips

    zip_data = c.acs5.state_zipcode(
        variables,
        state_fips=state_code,
        zcta='*',
    )
    city_data = c.acs5.state_place(
        variables,
        state_code,
        Census.ALL,
    )

    state_data = c.acs5.get(
        variables,
        {'for': 'state:*'},
    )
    state_df = pd.DataFrame(state_data)
    city_df = pd.DataFrame(city_data)
    zip_code_df = pd.DataFrame(zip_data)
    
    city_df = city_df.drop(['state','place'], axis=1)
    state_df = state_df.drop('state', axis=1)
    zip_code_df = zip_code_df.drop('NAME', axis=1)

    state_columns = {'NAME': 'state'}
    city_columns = {'NAME': 'city'}
    zip_columns = {'zip code tabulation area': 'zip_code'}

    state_df = state_df.rename(columns=state_columns)
    city_df = city_df.rename(columns=city_columns)
    zip_code_df = zip_code_df.rename(columns=zip_columns)

    city_df['city'] = city_df['city'].str.split(',', expand=True)[0]
    
    return state_df, city_df, zip_code_df



In [ ]:
year = 2021
state = 'California'

state_df, city_df, zip_code_df = extract_coordinates(year, state, api_key)

In [ ]:
state_df.head()

In [ ]:
city_df.head()

In [ ]:
zip_code_df.head()

In [6]:

def extract_coordinates_by_state(state):
    search = SearchEngine()
    zipcodes = search.by_state(state, returns=99999)  # Set a high value for returns

    geolocator = GoogleV3(api_key=g_key)

    coordinates = []
    for zipcode in zipcodes:
        location = geolocator.geocode(zipcode.zipcode)
        if location is not None:
            coordinates.append((zipcode.zipcode, location.latitude, location.longitude))

    return coordinates

# Usage example
state = 'California'
coordinates = extract_coordinates_by_state(state)

# Print the coordinates
for zipcode, lat, lng in coordinates:
    print(f"Zip Code: {zipcode}, Latitude: {lat}, Longitude: {lng}")


KeyboardInterrupt: 

In [ ]:
from geopy.geocoders import Nominatim
import concurrent.futures
import requests
import logging

In [ ]:
# pip install geopy

In [ ]:
# import pandas as pd
# from pyspark.sql import SparkSession
# import logging

# # Configure logging
# logging.basicConfig(level=logging.INFO)

# # Create a SparkSession
# spark = SparkSession.builder.getOrCreate()

# # Convert pandas DataFrame to PySpark DataFrame
# pyspark_df = spark.createDataFrame(zip_code_df)

# # Convert PySpark DataFrame to Pandas DataFrame
# pandas_df = pyspark_df.toPandas()

# # Add latitude and longitude columns using pandas apply method
# def add_coordinates(row):
#     zip_code = row['zip_code']
#     latitude, longitude = get_coordinates(zip_code, "")
#     print(f"Zip Code: {zip_code}, Latitude: {latitude}, Longitude: {longitude}")
#     return pd.Series([latitude, longitude])

# pandas_df[['latitude', 'longitude']] = pandas_df.apply(add_coordinates, axis=1)

# # Convert Pandas DataFrame back to PySpark DataFrame
# pyspark_df = spark.createDataFrame(pandas_df)

# # Show the resulting DataFrame
# pyspark_df.show()



In [ ]:
state_df['latitude'] = ""
state_df['longitude'] = ""
for index, row in state_df.iterrows():
    name = row['zip_code']
    latitude, longitude = get_coordinates(name, "")
    state_df.at[index, 'latitude'] = latitude
    state_df.at[index, 'longitude'] = longitude
    print(f"Coordinates for State {name}: Latitude={latitude}, Longitude={longitude}")


city_df['latitude'] = ""
city_df['longitude'] = ""
for index, row in city_df.iterrows():
    city = row['city']
    latitude, longitude = get_coordinates(city, "")
    city_df.at[index, 'latitude'] = latitude
    city_df.at[index, 'longitude'] = longitude
    print(f"Coordinates for city {city}: Latitude={latitude}, Longitude={longitude}")


zip_code_df['latitude'] = ""
zip_code_df['longitude'] = ""
for index, row in zip_code_df.iterrows():
    zip_code = row['zip_code']
    latitude, longitude = get_coordinates(zip_code, "")
    zip_code_df.at[index, 'latitude'] = latitude
    zip_code_df.at[index, 'longitude'] = longitude
    print(f"Coordinates for zip code {zip_code}: Latitude={latitude}, Longitude={longitude}")

# Transformation Notes
- add poverty_rate, employment_rate with pyspark in second ETL pipline to BQ warehouse

- split city column into city & state with PD first ETL pipeline to Storage lake 
- zip_df remove zip_code_tabluation_area first ETL pipeline with PD
- retrieve longitiude and latitude for zip, city, state level then add column and concat with PD
- drop 'place' column from city_df with PD
- reindex columns 


In [ ]:
# print(f'Columns: {state_df.dtypes}')
# print(f'Rows: {len(state_df)}')

In [ ]:
# def calculate_rates(df: pd.DataFrame) -> pd.DataFrame:
#     df["poverty_rate"] = 100 * df['poverty_count'].astype(int) / df["population"].astype(int)
#     df["unemployment_rate"] = 100 * df["unemployment_count"].astype(int) / df["population"].astype(int)

#     return df

# state_df = calculate_rates(state_df)
# city_df = calculate_rates(city_df)
# zip_code_df = calculate_rates(zip_code_df)

# state_df = state_df[['year', 'name', 'population', 'median_age', 'household_income', 'per_capita_income',
#                     'poverty_count', 'unemployment_count', 'employment_count', 'poverty_rate','unemployment_rate']]
# city_df = city_df[['year', 'city', 'population', 'median_age', 'household_income', 'per_capita_income',
#                 'poverty_count', 'unemployment_count', 'employment_count', 'poverty_rate', 'unemployment_rate']]
# zip_code_df = zip_code_df[['year', 'zip_code', 'population', 'median_age', 'household_income', 'per_capita_income',
#                         'poverty_count', 'unemployment_count', 'employment_count', 'poverty_rate', 'unemployment_rate']]


# State economic data

# City Economic Data

# Zip Code Economic Data


# State Geographic Data

In [ ]:
# values = zip_code_df['geographical_mobility'].isnull().sum()
# values

# Transforms - Geographic
- reorder index
    - year first, location second
- drop zip code tabulation area' 
- drop first half of zip code


# City Demographic Data

# Transformations

- Reorder year, place to start of df
- remove state # in state_df & city_df
- split city, and zip code column 